# IBMQ via Nexus

Here you can find some examples and explanations on using IBMQ via Nexus. These will require that you have set IBMQ credentials via Settings->Linked Accounts on the Nexus website.

There are two available configurations:

- `IBMQConfig` - for submitting projects to the IBMQ system
- `IBMQEmulatorConfig` - for submitting to an emulation of an IBMQ device (running on an `AerBackend` based off sampled device characteristics), but running in a Nexus server.

In [10]:
from datetime import datetime

from pytket import Circuit

import qnexus as qnx

In [ ]:
# See the names of the IBMQ devices available to you by default
qnx.devices.get_all(issuers=[qnx.devices.IssuerEnum.IBMQ]).df()

In [ ]:
# Alternatively if you would like to list devices available to you through a particular IBMQ plan, you can specify the hub, group and project
qnx.devices.get_all(
    issuers=[qnx.devices.IssuerEnum.IBMQ],
    ibmq_hub="my-hub",
    ibmq_group="my-group",
    ibmq_project="my-project",
).df()

## IBMQConfig

To use Nexus to run projects on IBMQ, you'll need to specify some configuration in a `IBMQConfig`.

The arguments you can provide to this are:

- `backend_name` (see Available Devices below)
- `hub` - The IBMQ hub you are running under
- `group` - The IBMQ group
- `project` - The IBMQ project
- `monitor` - Boolean specifying if you want to use the IBM job monitor. Defaults to True.

In [5]:
# An example configuration specifying the 127 qubit ibm_kyiv machine under the open plan
ibm_kyiv_configuration = qnx.IBMQConfig(
    backend_name="ibm_kyiv", hub="ibm-q", group="open", project="main"
)

# Use this configuration when submitting compile or execute jobs

## The IBMQ Emulator

To use Nexus to run projects on the IBMQ emulator, you'll need to specify some configuration in a `IBMQEmulatorConfig`.

The arguments you can provide to this are:

- `backend_name` - You can specify any available IBMQ backend
- `hub` - The IBMQ hub you are running under
- `group` - The IBMQ group
- `project` - The IBMQ project

In [6]:
# An example configuration specifying the 5 qubit ibm_nairobi machine (no longer available)
ibm_nairobi_emulator_configuration = qnx.IBMQEmulatorConfig(
    backend_name="ibm_nairobi", hub="ibm-q", group="open", project="main"
)

# Use this configuration when submitting compile or execute jobs

## Example of executing a circuit on ibm_kyiv

In [ ]:
my_project_ref = qnx.projects.get_or_create(name="My IBM Kyiv Project")

circuit = Circuit(2).H(0).CX(0, 1).measure_all()

my_circuit_ref = qnx.circuits.upload(
    name=f"My IBM Kyiv Circuit from {datetime.now()}",
    circuit=circuit,
    project=my_project_ref,
)

execute_job_ref = qnx.start_execute_job(
    circuits=[my_circuit_ref],
    name=f"My Execute Job from {datetime.now()}",
    n_shots=[100],
    backend_config=ibm_kyiv_configuration,
    project=my_project_ref,
)

qnx.jobs.wait_for(execute_job_ref)


# Retrieve a ExecutionResultRef for every Circuit that was executed
execute_job_result_refs = qnx.jobs.results(execute_job_ref)

# Get a pytket BackendResult for the execution
result = execute_job_result_refs[0].download_result()

result.get_counts()